## Scraping data about top 100 metal albums from this page: https://www.rollingstone.com/music/music-lists/the-100-greatest-metal-albums-of-all-time-113614/

## Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt

## The code

In [40]:
url = "https://www.rollingstone.com/music/music-lists/the-100-greatest-metal-albums-of-all-time-113614/"
s = Service('C:\webdrivers\chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(url)


WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//button[@id="onetrust-accept-btn-handler"]')))
cookies = driver.find_element(By.XPATH,'//button[@id="onetrust-accept-btn-handler"]')
cookies.click()
driver.refresh()
time.sleep(2)


albums_data = []


while len(albums_data) <= 100:
    albums = driver.find_elements(By.XPATH,'.//article[@class="c-gallery-vertical-album"]')
    for album in albums:
        album_rank = album.find_element(By.XPATH,'.//span[@class="c-gallery-vertical-album__number"]').text.strip()
        album_title = album.find_element(By.XPATH,'.//h2[@class="c-gallery-vertical-album__title"]').text.strip()
        album_desc = album.find_element(By.XPATH,'.//div[@class="pmc-not-a-paywall"]').text.strip()
        albums_data.append({"rank":album_rank,
                            "title":album_title,
                            "description":album_desc})
        if len(albums_data) == 50:
            load_more = driver.find_element(By.XPATH,'//div[@class="c-gallery-vertical__load-button"]//a')
            load_more = load_more.get_attribute('href')
            driver.get(load_more)
            break


## The data frame

In [92]:
df = pd.DataFrame(albums_data)

In [93]:
df

,rank,title,description
0,100,"Avenged Sevenfold, ‘City of Evil’ (2005)",Avenged Sevenfold’s third full-length was an i...
1,99,"Evanescence, ‘Fallen’ (2003)","From the moment the haunting, moody and cinema..."
2,98,"Sunn O))), ‘Monoliths & Dimensions’ (2009)",“I would never claim that Sunn O))) is a jazz ...
3,97,"Gojira, ‘From Mars to Sirius’ (2005)",Gojira emerged in the early 2000s – a time dom...
4,96,"Kvelertak, ‘Meir’ (2013)",Metal in the 2010s is almost absurdly balkaniz...
...,...,...,...
145,5,"Black Sabbath, ‘Black Sabbath’ (1970)",A few years after guitarists first started cra...
146,4,"Iron Maiden, ‘The Number of the Beast’ (1982)",By the time Iron Maiden hit the studio with ve...
147,3,"Judas Priest, ‘British Steel’ (1980)","In the Seventies, British metal – the down-tun..."
148,2,"Metallica, ‘Master of Puppets’ (1986)",It begins like a Western with ominous acoustic...


In [94]:
def add_columns(df):
    df=df.copy()
    df['album'] = df['title'].apply(lambda x: x.split('‘')[1].split('’')[0])
    df['year'] = df['title'].apply(lambda x: x.split('(')[1].replace(')',''))
    df['band'] = df['title'].apply(lambda x: x.split(',')[0])
    df = df.loc[:,~df.columns.isin(['title'])]
    return df

In [96]:
df = add_columns(df)

In [97]:
df

,rank,description,album,year,band
0,100,Avenged Sevenfold’s third full-length was an i...,City of Evil,2005,Avenged Sevenfold
1,99,"From the moment the haunting, moody and cinema...",Fallen,2003,Evanescence
2,98,“I would never claim that Sunn O))) is a jazz ...,Monoliths & Dimensions,2009,Sunn O)))
3,97,Gojira emerged in the early 2000s – a time dom...,From Mars to Sirius,2005,Gojira
4,96,Metal in the 2010s is almost absurdly balkaniz...,Meir,2013,Kvelertak
...,...,...,...,...,...
145,5,A few years after guitarists first started cra...,Black Sabbath,1970,Black Sabbath
146,4,By the time Iron Maiden hit the studio with ve...,The Number of the Beast,1982,Iron Maiden
147,3,"In the Seventies, British metal – the down-tun...",British Steel,1980,Judas Priest
148,2,It begins like a Western with ominous acoustic...,Master of Puppets,1986,Metallica
